# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [59]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [20]:
# all import statements needed for the project, for example:

import os

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import re
import geopandas as gpd
import math
import glob
import numpy as np
import sqlite3
import sqlalchemy as db
from sqlalchemy import create_engine

In [4]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

PARQUET_FILES = "parquet_files"
TAXI_ZONES_DIR = "taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = "weather_data"

#CRS = 4326  # coordinate reference system

# (lat, lon)
#NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [5]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Download All data

In [25]:
def get_all_urls_from_tlc_page():
    response = requests.get(TLC_URL)
    html = response.content
    return html

In [26]:
#Identifies all of the yellow and fhvhv parquet files for years 2020 - 2024
pattern = re.compile(r".*(yellow|fhvhv).*(2020|2021|2022|2023|2024)-\d{2}\.parquet")

def filter_parquet_urls():
    html = get_all_urls_from_tlc_page()
    soup = bs4.BeautifulSoup(html, "html.parser")
    urls = soup.find_all("a", href=pattern)
    parquet_urls = [link["href"].strip() for link in urls]
    return parquet_urls

In [27]:
folder_name = "parquet_files"

# Check if the folder exists
if not os.path.exists(folder_name):
    os.mkdir(folder_name)
    print(f"Folder '{folder_name}' created successfully!")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'parquet_files' already exists.


In [28]:
# parses the filename from the link and then downloads the files one by one
def download_parquet_files():
    for link in filter_parquet_urls():
        filename = link.split("/")[-1]
        r = requests.get(link)
        with open(f"parquet_files/{filename}", "wb") as f:
            f.write(r.content)

#run the first time to download data
#download_parquet_files()

### Load Taxi Zones & Parquet Files

In [47]:
#Reads the shape file
def load_taxi_zones(shapefile):
    taxi_zones = gpd.read_file(shapefile)
    return taxi_zones

In [48]:
gdf_taxi_zones = load_taxi_zones(TAXI_ZONES_SHAPEFILE)
print(gdf_taxi_zones.head())

   OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0         1    0.116357    0.000782           Newark Airport           1   
1         2    0.433470    0.004866              Jamaica Bay           2   
2         3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3         4    0.043567    0.000112            Alphabet City           4   
4         5    0.092146    0.000498            Arden Heights           5   

         borough                                           geometry  
0            EWR  POLYGON ((933100.918 192536.086, 933091.011 19...  
1         Queens  MULTIPOLYGON (((1033269.244 172126.008, 103343...  
2          Bronx  POLYGON ((1026308.77 256767.698, 1026495.593 2...  
3      Manhattan  POLYGON ((992073.467 203714.076, 992068.667 20...  
4  Staten Island  POLYGON ((935843.31 144283.336, 936046.565 144...  


In [49]:
# converts taxi zone geometry coordinates to the appropriate coordinate system  
gdf_taxi_zones = gdf_taxi_zones.to_crs(epsg=4326)
gdf_taxi_zones

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.695, -74.18449 40.6951,..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((-73.85107 40.91037, -73.85207 40.909..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((-73.90175 40.76078, -73.90147 40.759..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((-74.01333 40.70503, -74.01327 40.704..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((-73.94383 40.78286, -73.94376 ..."


In [50]:
# load parquet file into a pandas DataFrame
def load_parquet_file(file_path):
    df = pd.read_parquet(file_path)
    return df

In [51]:
# load a random yellow taxi trip parquet file to check if the function works correctly for testing purposes
example = os.path.join(PARQUET_FILES, "yellow_tripdata_2023-01.parquet")
example_df = load_parquet_file(example)

# preview the data
print(example_df.head())

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1         2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2         2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3         1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4         2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           0.97         1.0                  N           161           141   
1           1.10         1.0                  N            43           237   
2           2.51         1.0                  N            48           238   
3           1.90         1.0                  N           138             7   
4           1.43         1.0                  N           107            79   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [11]:
# load a random High-Volume For-Hire Vehicle trip parquet file to check if the function works correctly
example2 = os.path.join(PARQUET_FILES, "fhvhv_tripdata_2023-01.parquet")
example_df2 = load_parquet_file(example2)

# preview the data
print(example_df2.head())

  hvfhs_license_num dispatching_base_num originating_base_num  \
0            HV0003               B03404               B03404   
1            HV0003               B03404               B03404   
2            HV0003               B03404               B03404   
3            HV0003               B03404               B03404   
4            HV0003               B03404               B03404   

     request_datetime   on_scene_datetime     pickup_datetime  \
0 2023-01-01 00:18:06 2023-01-01 00:19:24 2023-01-01 00:19:38   
1 2023-01-01 00:48:42 2023-01-01 00:56:20 2023-01-01 00:58:39   
2 2023-01-01 00:15:35 2023-01-01 00:20:14 2023-01-01 00:20:27   
3 2023-01-01 00:35:24 2023-01-01 00:39:30 2023-01-01 00:41:05   
4 2023-01-01 00:43:15 2023-01-01 00:51:10 2023-01-01 00:52:47   

     dropoff_datetime  PULocationID  DOLocationID  trip_miles  ...  sales_tax  \
0 2023-01-01 00:48:07            48            68        0.94  ...       2.30   
1 2023-01-01 01:33:08           246           163       

In [12]:
example_df2.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [13]:
example_df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

### Cleaning and Filtering
* Remove all non-Uber data from fhvhv
* Remove all invalid pickup and dropoff location IDs for both uber and yellow taxi, where ID is greater than 263 using the `shp` file
* Remove unnecessary columns and only keeping columns needed to answer questions in the other parts of this project
* Remove invalid data points (use your discretion!)
* normalize column names; 
normalieg and using appropriate column types for the respective dat

* Remove trips from both uber and yellow taxi that start and/or end outside of the following latitude/longitude coordinate box: (40.560445, -74.242330) and (40.908524, -73.71704).


In [52]:
#Compute the center of the taxi zones for easier comparison and adds a column to the df of our shapefile 
gdf_taxi_zones['centroid'] = gdf_taxi_zones.geometry.centroid

#Rename 'LocationID' to 'location_id' for consistency
gdf_taxi_zones = gdf_taxi_zones.rename(columns={'LocationID': 'location_id'})

#Removes the bulky geometry column after using it to compute centroid. 
gdf_taxi_zones = gdf_taxi_zones[['zone', 'location_id', 'centroid']]

C:\Users\huang\AppData\Local\Temp\ipykernel_15944\103723099.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_taxi_zones['centroid'] = gdf_taxi_zones.geometry.centroid


In [53]:
gdf_taxi_zones

,zone,location_id,centroid
0,Newark Airport,1,POINT (-74.174 40.69183)
1,Jamaica Bay,2,POINT (-73.8313 40.61675)
2,Allerton/Pelham Gardens,3,POINT (-73.84742 40.86447)
3,Alphabet City,4,POINT (-73.97697 40.72375)
4,Arden Heights,5,POINT (-74.18848 40.55266)
...,...,...,...
258,Woodlawn/Wakefield,259,POINT (-73.85222 40.89793)
259,Woodside,260,POINT (-73.90631 40.74423)
260,World Trade Center,261,POINT (-74.01302 40.70914)
261,Yorkville East,262,POINT (-73.94651 40.77593)


### Calculate Sample Size

In [54]:
# default: 95% confidence interval, 5% margin of error, p of 0.5 (estimated) proportion of the population which has the attribute in question
def cochran_sample_size(population_size):
    z_score=1.96
    margin_of_error=0.05
    p=0.5
    sample_size = ((z_score**2)*p*(1-p)) / (margin_of_error**2)
    adjusted_sample_size = sample_size / (1 + ((sample_size-1)/population_size))

    return int(adjusted_sample_size)

### Common Functions

In [55]:
#Filter function to remove unecessary rows
def filter_data(data):
    #Ensure PU and DO locations are within valid location IDs (<= 263)
    data = data[(data['pickup_location_id'] <= 263) & (data['dropoff_location_id'] <= 263)]
    #Filters out rides where pickup and dropoff locations are the same
    filtered_data = data[data['trip_distance'] != 0]
        
    return filtered_data

In [56]:
# Removes trips from both uber and yellow taxi that start and/or end outside of the following latitude/longitude coordinate box:
def find_centroid(data):
    LAT_MIN, LON_MIN = 40.560445, -74.242330
    LAT_MAX, LON_MAX = 40.908524, -73.717047
    
    # Extract latitude and longitude from the 'centroid' column using .apply()
    data['centroid_lat'] = data['centroid'].apply(lambda point: point.y)
    data['centroid_lon'] = data['centroid'].apply(lambda point: point.x)
    
    # Filter rows where the centroid coordinates are within the bounding coordinate box
    centroid_data = data[
        (data['centroid_lat'] >= LAT_MIN) & (data['centroid_lat'] <= LAT_MAX) &
        (data['centroid_lon'] >= LON_MIN) & (data['centroid_lon'] <= LON_MAX)
    ]
    return centroid_data

### Process Taxi Data

In [57]:
#Grab all of the parquet files in the directory. glob.glob is used to identify/match the pattern, path.join retrieves all the paths 
all_taxi_parquet_files = glob.glob(os.path.join(PARQUET_FILES, "*yellow*.parquet"))

In [62]:
taxi_columns_mapping = {
    'tpep_pickup_datetime': 'pickup_datetime',
    'tpep_dropoff_datetime': 'dropoff_datetime',
    'extra': 'rush_hour_surcharge',
    'PULocationID': 'pickup_location_id',
    'DOLocationID': 'dropoff_location_id',
     'fare_amount': 'base_passenger_fare',
    'RatecodeID': 'rate_code_id'
}

#Make a list of just the columns we need for analysis
columns_to_keep = [
    'pickup_datetime', 'dropoff_datetime', 'trip_distance', 'rate_code_id',
    'pickup_location_id', 'dropoff_location_id', 'base_passenger_fare', 'rush_hour_surcharge', 'mta_tax',
    'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
    'congestion_surcharge', 'airport_fee'
]



#Create samples of all taxi parquet files according to cochran's sample size formula. Later, we concatenate all sample dfs into one df. 
sampled_taxi_dfs = []

for file_path in all_taxi_parquet_files:      
    taxi_df = load_parquet_file(file_path) #Makes a df for every parquet file 
    taxi_df = taxi_df.rename(columns=taxi_columns_mapping)
    
    population_size = len(taxi_df)
    sample_size = cochran_sample_size(population_size)
    sampled_taxi_df = taxi_df.sample(n=sample_size, random_state=42)
    #We found that there were a few files that did not have airport_fee as a column. We populate airport_fee with NaN for such parquet files.
    for col in columns_to_keep:  
        if col not in sampled_taxi_df.columns:
            sampled_taxi_df[col] = np.nan 
    sampled_taxi_df = sampled_taxi_df[columns_to_keep]
    sampled_taxi_dfs.append(sampled_taxi_df)

    # create one gigantic dataframe with data from every month needed
sampled_taxi_data = pd.concat(sampled_taxi_dfs)

sampled_taxi_data = filter_data(sampled_taxi_data)

# Make a single df that includes the taxi rides and their corresponding coordinates by merging the shape file with the ride files.
final_taxi_data = pd.merge(sampled_taxi_data, gdf_taxi_zones, left_on = 'pickup_location_id', right_on = 'location_id', how="inner")

final_taxi_data = find_centroid(final_taxi_data)

C:\Users\huang\AppData\Local\Temp\ipykernel_15944\4192337244.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sampled_taxi_data = pd.concat(sampled_taxi_dfs)


In [63]:
final_taxi_data.head()

,pickup_datetime,dropoff_datetime,trip_distance,rate_code_id,pickup_location_id,dropoff_location_id,base_passenger_fare,rush_hour_surcharge,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,zone,location_id,centroid,centroid_lat,centroid_lon
0,2020-01-25 10:49:58,2020-01-25 11:07:35,3.28,1.0,142,246,14.0,0.0,0.5,1.70,0.0,0.3,19.00,2.5,NaN,Lincoln Square East,142,POINT (-73.98153 40.77363),40.773633,-73.981532
1,2020-01-15 07:30:08,2020-01-15 07:40:01,1.75,1.0,238,166,8.5,0.0,0.5,1.20,0.0,0.3,13.00,2.5,NaN,Upper West Side North,238,POINT (-73.97305 40.7917),40.791705,-73.973049
2,2020-01-09 06:29:09,2020-01-09 06:35:44,0.87,1.0,100,164,5.5,0.0,0.5,0.00,0.0,0.3,8.80,2.5,NaN,Garment District,100,POINT (-73.98879 40.75351),40.753513,-73.988787
3,2020-01-26 12:24:04,2020-01-26 12:29:15,0.98,1.0,161,43,5.5,0.0,0.5,0.00,0.0,0.3,8.80,2.5,NaN,Midtown Center,161,POINT (-73.9777 40.75803),40.758028,-73.977698
4,2020-01-30 07:57:53,2020-01-30 08:10:19,1.30,1.0,229,262,9.0,2.5,0.5,2.45,0.0,0.3,14.75,2.5,NaN,Sutton Place/Turtle Bay North,229,POINT (-73.96515 40.75673),40.756729,-73.965146


In [64]:
final_taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20826 entries, 0 to 20828
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   pickup_datetime        20826 non-null  datetime64[us]
 1   dropoff_datetime       20826 non-null  datetime64[us]
 2   trip_distance          20826 non-null  float64       
 3   rate_code_id           19770 non-null  float64       
 4   pickup_location_id     20826 non-null  int64         
 5   dropoff_location_id    20826 non-null  int64         
 6   base_passenger_fare    20826 non-null  float64       
 7   rush_hour_surcharge    20826 non-null  float64       
 8   mta_tax                20826 non-null  float64       
 9   tip_amount             20826 non-null  float64       
 10  tolls_amount           20826 non-null  float64       
 11  improvement_surcharge  20826 non-null  float64       
 12  total_amount           20826 non-null  float64       
 13  conges

In [65]:
final_taxi_data.describe()

,pickup_datetime,dropoff_datetime,trip_distance,rate_code_id,pickup_location_id,dropoff_location_id,base_passenger_fare,rush_hour_surcharge,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,location_id,centroid_lat,centroid_lon
count,20826,20826,20826.000000,19770.000000,20826.000000,20826.000000,20826.000000,20826.000000,20826.000000,20826.000000,20826.000000,20826.000000,20826.000000,19770.000000,7947.000000,20826.000000,20826.000000,20826.000000
mean,2022-05-01 10:17:07.596274,2022-05-01 10:33:09.629309,3.275384,1.415023,164.151157,160.661001,15.153432,1.205869,0.490733,2.680401,0.431877,0.543945,22.188515,2.297547,0.086825,164.151157,40.753285,-73.966859
min,2020-01-01 00:11:06,2020-01-01 00:30:50,0.010000,1.000000,4.000000,1.000000,-81.520000,-7.500000,-0.500000,0.000000,-34.200000,-1.000000,-108.200000,-2.500000,-1.250000,4.000000,40.576961,-74.029892
25%,2021-02-28 21:30:14.250000,2021-02-28 21:36:58.750000,1.090000,1.000000,132.000000,107.000000,7.200000,0.000000,0.500000,0.000000,0.000000,0.300000,12.600000,2.500000,0.000000,132.000000,40.740337,-73.989845
50%,2022-04-30 15:36:52.500000,2022-04-30 16:06:25,1.800000,1.000000,162.000000,161.000000,10.700000,0.500000,0.500000,2.160000,0.000000,0.300000,16.800000,2.500000,0.000000,162.000000,40.758028,-73.977698
75%,2023-06-29 14:40:03,2023-06-29 14:50:15.250000,3.310000,1.000000,234.000000,234.000000,17.000000,2.500000,0.500000,3.440000,0.000000,1.000000,24.360000,2.500000,0.000000,234.000000,40.773633,-73.961764
max,2024-08-31 22:43:47,2024-08-31 23:26:23,67.900000,99.000000,263.000000,263.000000,209.500000,11.750000,0.500000,50.000000,40.000000,1.000000,262.700000,2.500000,1.250000,263.000000,40.899529,-73.739337
std,NaN,NaN,4.121171,6.028092,65.215425,70.490221,13.948344,1.512793,0.089069,3.157641,1.825400,0.351174,17.759500,0.735558,0.321505,65.215425,0.032466,0.045223


### Processing Uber Data

In [66]:
#Grab all of the parquet files in the directory. glob.glob is used to identify/match the pattern, path.join retrieves all the paths 
all_fhvhv_parquet_files = glob.glob(os.path.join(PARQUET_FILES, "*fhvhv*.parquet"))

In [68]:
uber_columns_mapping = {
    'trip_miles': 'trip_distance',
    'PULocationID': 'pickup_location_id',
    'DOLocationID': 'dropoff_location_id',
    'tolls': 'tolls_amount',
    'tips': 'tip_amount',
    'bcf': 'black_car_fund_fee',
    
}

#Create samples of all uber parquet files according to cochran's sample size formula. Later, we concatenate all sample dfs into one df. 
sampled_uber_dfs = []
columns_to_keep = ['hvfhs_license_num',
       'request_datetime', 'pickup_datetime',
       'dropoff_datetime', 'pickup_location_id', 'dropoff_location_id', 'trip_distance',
        'base_passenger_fare', 'tolls_amount', 'black_car_fund_fee', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tip_amount']

for file_path in all_fhvhv_parquet_files:      
    uber_df = load_parquet_file(file_path) #Makes a df for every parquet file 
    uber_df = uber_df.rename(columns=uber_columns_mapping)
    uber_df = uber_df[uber_df['hvfhs_license_num'] == 'HV0003'] #Filters out non-uber rides from the hvfhs files before creating samples
    population_size = len(uber_df)
    sample_size = cochran_sample_size(population_size)
    sampled_uber_df = uber_df.sample(n=sample_size, random_state=42)
    sampled_uber_df = sampled_uber_df[columns_to_keep]
    sampled_uber_dfs.append(sampled_uber_df)

    # create one gigantic dataframe with data from every month needed
sampled_uber_data = pd.concat(sampled_uber_dfs)

sampled_uber_data = filter_data(sampled_uber_data)

# Make a single df that includes the taxi rides and their corresponding coordinates by merging the shape file with the ride files.
final_uber_data = pd.merge(sampled_uber_data, gdf_taxi_zones, left_on = 'pickup_location_id', right_on = 'location_id', how="inner")

final_uber_data = find_centroid(final_uber_data)

C:\Users\huang\AppData\Local\Temp\ipykernel_15944\3122417029.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sampled_uber_data = pd.concat(sampled_uber_dfs)


In [69]:
final_uber_data.head()

,hvfhs_license_num,request_datetime,pickup_datetime,dropoff_datetime,pickup_location_id,dropoff_location_id,trip_distance,base_passenger_fare,tolls_amount,black_car_fund_fee,sales_tax,congestion_surcharge,airport_fee,tip_amount,zone,location_id,centroid,centroid_lat,centroid_lon
0,HV0003,2020-01-26 22:19:19,2020-01-26 22:21:28,2020-01-26 22:44:27,48,80,6.95,27.22,6.12,0.83,2.96,2.75,NaN,0.0,Clinton East,48,POINT (-73.98984 40.76225),40.762253,-73.989845
1,HV0003,2020-01-19 15:28:56,2020-01-19 15:34:47,2020-01-19 16:00:12,114,138,10.78,32.52,0.00,0.81,2.89,2.75,NaN,0.0,Greenwich Village South,114,POINT (-73.99738 40.72834),40.728340,-73.997380
2,HV0003,2020-01-25 20:55:33,2020-01-25 20:58:45,2020-01-25 21:04:11,61,225,1.00,7.19,0.00,0.18,0.64,0.00,NaN,0.0,Crown Heights North,61,POINT (-73.93929 40.67447),40.674469,-73.939287
3,HV0003,2020-01-11 08:39:09,2020-01-11 08:42:05,2020-01-11 08:49:12,91,39,1.95,5.78,0.00,0.14,0.51,0.00,NaN,0.0,Flatlands,91,POINT (-73.9301 40.62627),40.626273,-73.930097
4,HV0003,2020-01-19 18:13:08,2020-01-19 18:17:31,2020-01-19 18:34:47,256,52,4.23,17.65,0.00,0.36,1.18,0.00,NaN,1.0,Williamsburg (South Side),256,POINT (-73.9599 40.71088),40.710880,-73.959905


In [70]:
final_uber_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20678 entries, 0 to 20731
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hvfhs_license_num     20678 non-null  object        
 1   request_datetime      20678 non-null  datetime64[us]
 2   pickup_datetime       20678 non-null  datetime64[us]
 3   dropoff_datetime      20678 non-null  datetime64[us]
 4   pickup_location_id    20678 non-null  int64         
 5   dropoff_location_id   20678 non-null  int64         
 6   trip_distance         20678 non-null  float64       
 7   base_passenger_fare   20678 non-null  float64       
 8   tolls_amount          20678 non-null  float64       
 9   black_car_fund_fee    20678 non-null  float64       
 10  sales_tax             20678 non-null  float64       
 11  congestion_surcharge  20678 non-null  float64       
 12  airport_fee           15107 non-null  float64       
 13  tip_amount           

In [71]:
final_uber_data.describe()

,request_datetime,pickup_datetime,dropoff_datetime,pickup_location_id,dropoff_location_id,trip_distance,base_passenger_fare,tolls_amount,black_car_fund_fee,sales_tax,congestion_surcharge,airport_fee,tip_amount,location_id,centroid_lat,centroid_lon
count,20678,20678,20678,20678.000000,20678.000000,20678.000000,20678.000000,20678.000000,20678.000000,20678.000000,20678.000000,15107.000000,20678.000000,20678.000000,20678.000000,20678.000000
mean,2022-04-29 03:51:06.178257,2022-04-29 03:55:45.584002,2022-04-29 04:13:38.630138,137.800077,137.759745,4.439277,21.201442,0.656449,0.620884,1.895659,1.052326,0.191468,0.820059,137.800077,40.737434,-73.934434
min,2020-01-01 01:14:43,2020-01-01 01:21:23,2020-01-01 01:46:38,3.000000,1.000000,0.020000,-9.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,40.561994,-74.170887
25%,2021-02-24 10:25:41.250000,2021-02-24 10:29:33.250000,2021-02-24 10:49:42.250000,73.000000,74.000000,1.540000,10.490000,0.000000,0.290000,0.910000,0.000000,0.000000,0.000000,73.000000,40.690787,-73.984196
50%,2022-04-25 21:30:18,2022-04-25 21:33:31.500000,2022-04-25 21:42:09,139.000000,138.000000,2.810000,16.720000,0.000000,0.470000,1.470000,0.000000,0.000000,0.000000,139.000000,40.737699,-73.948789
75%,2023-06-30 11:11:37,2023-06-30 11:14:15,2023-06-30 11:23:13.250000,211.000000,209.000000,5.710000,26.570000,0.000000,0.770000,2.380000,2.750000,0.000000,0.000000,211.000000,40.774376,-73.898956
max,2024-08-31 23:43:36,2024-08-31 23:49:13,2024-09-01 00:10:26,263.000000,263.000000,42.660000,255.840000,43.910000,7.950000,23.510000,2.750000,5.000000,40.000000,263.000000,40.899529,-73.726655
std,NaN,NaN,NaN,75.489021,75.367212,4.359484,15.753046,2.520313,0.500625,1.450202,1.332477,0.670128,2.429705,75.489021,0.069218,0.065377


### Processing Weather Data

In [10]:
def get_all_weather_csvs(directory):
    weather_dfs = []

    # Iterate over all files in the given directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path, low_memory=False)
        # Append the DataFrame to the list
        weather_dfs.append(df)
    weather_dfs = pd.concat(weather_dfs, ignore_index=True)
    return weather_dfs


In [11]:
columns_to_keep = ['DATE','LATITUDE', 'LONGITUDE', 'MonthlyTotalLiquidPrecipitation', 'DailyPrecipitation', 'HourlyPrecipitation', 'DailyAverageWindSpeed', 'HourlyWindSpeed', 'DailySnowfall']

weather_data = get_all_weather_csvs(WEATHER_CSV_DIR)
weather_data = weather_data[columns_to_keep]
weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])

# weather_data[weather_data["DailyPrecipitation"].isna()] #54343
weather_data[weather_data["HourlyPrecipitation"].isna()] #8,523
# weather_data


,DATE,LATITUDE,LONGITUDE,MonthlyTotalLiquidPrecipitation,DailyPrecipitation,HourlyPrecipitation,DailyAverageWindSpeed,HourlyWindSpeed,DailySnowfall
24,2020-01-01 23:59:00,40.77898,-73.96925,NaN,0.00,NaN,8.6,NaN,0.0
29,2020-01-02 04:51:00,40.77898,-73.96925,NaN,NaN,NaN,NaN,8.0,NaN
30,2020-01-02 05:51:00,40.77898,-73.96925,NaN,NaN,NaN,NaN,6.0,NaN
31,2020-01-02 06:51:00,40.77898,-73.96925,NaN,NaN,NaN,NaN,5.0,NaN
32,2020-01-02 07:51:00,40.77898,-73.96925,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...
56093,2024-10-22 14:51:00,40.77898,-73.96925,NaN,NaN,NaN,NaN,3.0,NaN
56094,2024-10-22 15:51:00,40.77898,-73.96925,NaN,NaN,NaN,NaN,0.0,NaN
56095,2024-10-22 16:51:00,40.77898,-73.96925,NaN,NaN,NaN,NaN,0.0,NaN
56096,2024-10-22 17:51:00,40.77898,-73.96925,NaN,NaN,NaN,NaN,0.0,NaN


In [12]:
#We needed to do this because an error was being thrown that the hourly precipitation values were strings. 
# Identify rows where HourlyPrecipitation is a string
string_precipitation_rows = weather_data[weather_data['HourlyPrecipitation'].apply(lambda x: isinstance(x, str))]

# Print or display the rows to inspect the problematic values
print(string_precipitation_rows)
# Identify rows where HourlyPrecipitation is a string
string_precipitation_rows = weather_data[weather_data['HourlyPrecipitation'].apply(lambda x: isinstance(x, str))]

# Extract the HourlyPrecipitation values that are strings
string_values = string_precipitation_rows['HourlyPrecipitation']

# Get unique values and their count using value_counts()
string_value_counts = string_values.value_counts()

# Print or display the unique string values and their count
print("Unique String Values and Their Counts in HourlyPrecipitation:")
print(string_value_counts)


                     DATE  LATITUDE  LONGITUDE  \
0     2020-01-01 00:51:00  40.77898  -73.96925   
1     2020-01-01 01:51:00  40.77898  -73.96925   
2     2020-01-01 02:51:00  40.77898  -73.96925   
3     2020-01-01 03:51:00  40.77898  -73.96925   
4     2020-01-01 04:51:00  40.77898  -73.96925   
...                   ...       ...        ...   
56073 2024-10-21 19:51:00  40.77898  -73.96925   
56074 2024-10-21 20:51:00  40.77898  -73.96925   
56075 2024-10-21 21:51:00  40.77898  -73.96925   
56076 2024-10-21 22:51:00  40.77898  -73.96925   
56077 2024-10-21 23:51:00  40.77898  -73.96925   

      MonthlyTotalLiquidPrecipitation DailyPrecipitation HourlyPrecipitation  \
0                                 NaN                NaN                0.00   
1                                 NaN                NaN                0.00   
2                                 NaN                NaN                0.00   
3                                 NaN                NaN                0.00   

In [13]:
#Cleans out the string values based on different cases identified in the query above
# Replace 'T' (trace amount) with 0.00
weather_data['HourlyPrecipitation'].replace('T', 0.00, inplace=True)

# Function to clean non-numeric characters from values
def clean_precipitation_value(value):
    if isinstance(value, str):
        # Remove all non-numeric characters except the decimal point
        return re.sub(r'[^0-9.]', '', value)
    return value

# Apply the cleaning function to the HourlyPrecipitation column
weather_data['HourlyPrecipitation'] = weather_data['HourlyPrecipitation'].apply(clean_precipitation_value)

# Convert all cleaned values to numeric, coercing any problematic ones to NaN
weather_data['HourlyPrecipitation'] = pd.to_numeric(weather_data['HourlyPrecipitation'], errors='coerce')

# Display the cleaned DataFrame
print(weather_data.head())

                 DATE  LATITUDE  LONGITUDE MonthlyTotalLiquidPrecipitation  \
0 2020-01-01 00:51:00  40.77898  -73.96925                             NaN   
1 2020-01-01 01:51:00  40.77898  -73.96925                             NaN   
2 2020-01-01 02:51:00  40.77898  -73.96925                             NaN   
3 2020-01-01 03:51:00  40.77898  -73.96925                             NaN   
4 2020-01-01 04:51:00  40.77898  -73.96925                             NaN   

  DailyPrecipitation  HourlyPrecipitation  DailyAverageWindSpeed  \
0                NaN                  0.0                    NaN   
1                NaN                  0.0                    NaN   
2                NaN                  0.0                    NaN   
3                NaN                  0.0                    NaN   
4                NaN                  0.0                    NaN   

   HourlyWindSpeed DailySnowfall  
0              8.0           NaN  
1              8.0           NaN  
2             14.

C:\Users\huang\AppData\Local\Temp\ipykernel_15944\2630901844.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weather_data['HourlyPrecipitation'].replace('T', 0.00, inplace=True)


In [14]:
# Fill in the remaining null hourly precipiation values 
# Ensure that 'DATE' is in the correct datetime format
weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])

# Add a column for just the date (without time), which will help with grouping
weather_data['DATE_ONLY'] = weather_data['DATE'].dt.date
# Group by each unique date
daily_groups = weather_data.groupby('DATE_ONLY')

# Loop over each group of a specific date and calculate the remaining precipitation to distribute
for date, group in daily_groups:
    # Get the daily precipitation for the current day
    daily_precipitation = group['DailyPrecipitation'].iloc[0]
    
    # Calculate the sum of existing hourly precipitation values (if any)
    existing_hourly_precip = group['HourlyPrecipitation'].sum(skipna=True)

    # Calculate the remaining precipitation that needs to be distributed to missing hours
    remaining_precip = daily_precipitation - existing_hourly_precip

    # Get the number of hours with missing HourlyPrecipitation
    missing_hours = group['HourlyPrecipitation'].isna().sum()

    # Calculate how much to distribute to each missing hour
    if missing_hours > 0 and remaining_precip > 0:
        hourly_precipitation_to_assign = remaining_precip / missing_hours
    else:
        hourly_precipitation_to_assign = 0

    # Fill missing HourlyPrecipitation values with the calculated amount
    weather_data.loc[group.index, 'HourlyPrecipitation'] = group['HourlyPrecipitation'].fillna(hourly_precipitation_to_assign)


In [106]:
#Confirm there are no null hourly precipitation values anymore 
weather_data[weather_data["HourlyPrecipitation"].isna()] #0 

,DATE,LATITUDE,LONGITUDE,MonthlyTotalLiquidPrecipitation,DailyPrecipitation,HourlyPrecipitation,DailyAverageWindSpeed,HourlyWindSpeed,DailySnowfall,DATE_ONLY


In [15]:
weather_data.head()

,DATE,LATITUDE,LONGITUDE,MonthlyTotalLiquidPrecipitation,DailyPrecipitation,HourlyPrecipitation,DailyAverageWindSpeed,HourlyWindSpeed,DailySnowfall,DATE_ONLY
0,2020-01-01 00:51:00,40.77898,-73.96925,NaN,NaN,0.0,NaN,8.0,NaN,2020-01-01
1,2020-01-01 01:51:00,40.77898,-73.96925,NaN,NaN,0.0,NaN,8.0,NaN,2020-01-01
2,2020-01-01 02:51:00,40.77898,-73.96925,NaN,NaN,0.0,NaN,14.0,NaN,2020-01-01
3,2020-01-01 03:51:00,40.77898,-73.96925,NaN,NaN,0.0,NaN,11.0,NaN,2020-01-01
4,2020-01-01 04:51:00,40.77898,-73.96925,NaN,NaN,0.0,NaN,6.0,NaN,2020-01-01


In [76]:
weather_columns_mapping = {
 'DATE': 'date',
 'LATITUDE': 'latitude',
 'LONGITUDE': 'longitude',
 'DailyPrecipitation': 'daily_precipitation',
 'HourlyPrecipitation': 'hourly_precipitation',
 'DailyAverageWindSpeed': 'daily_average_wind_speed',
 'HourlyWindSpeed': 'hourly_wind_speed',
 'DailySnowfall': 'daily_snowfall'
}
weather_data.rename(columns=weather_columns_mapping, inplace=True)

In [78]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56098 entries, 0 to 56097
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   date                             56098 non-null  datetime64[ns]
 1   latitude                         56098 non-null  float64       
 2   longitude                        56098 non-null  float64       
 3   MonthlyTotalLiquidPrecipitation  52 non-null     object        
 4   daily_precipitation              1755 non-null   object        
 5   hourly_precipitation             56098 non-null  float64       
 6   daily_average_wind_speed         1697 non-null   float64       
 7   hourly_wind_speed                49660 non-null  float64       
 8   daily_snowfall                   1750 non-null   object        
 9   DATE_ONLY                        56098 non-null  object        
dtypes: datetime64[ns](1), float64(5), object(4)
memory usage: 

In [79]:
weather_data.describe()

,date,latitude,longitude,hourly_precipitation,daily_average_wind_speed,hourly_wind_speed
count,56098,5.609800e+04,5.609800e+04,56098.000000,1697.000000,49660.000000
mean,2022-05-29 21:14:19.618881024,4.077898e+01,-7.396925e+01,0.010511,5.000766,5.125453
min,2020-01-01 00:51:00,4.077898e+01,-7.396925e+01,0.000000,0.600000,0.000000
25%,2021-03-18 19:01:45,4.077898e+01,-7.396925e+01,0.000000,3.200000,3.000000
50%,2022-05-28 01:21:00,4.077898e+01,-7.396925e+01,0.000000,4.600000,5.000000
75%,2023-08-15 05:39:00,4.077898e+01,-7.396925e+01,0.000000,6.300000,7.000000
max,2024-10-22 18:51:00,4.077898e+01,-7.396925e+01,3.470000,14.200000,2237.000000
std,NaN,4.214267e-11,5.815134e-11,0.056783,2.339258,14.653212


## Part 2: Storing Cleaned Data

In [21]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
    date DATETIME,
    latitude REAL,
    longitude REAL,
    hourly_precipitation REAL,
    hourly_wind_speed REAL,
    daily_snowfall REAL
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
    date DATETIME,
    latitude REAL,
    longitude REAL,
    daily_precipitation REAL,
    daily_wind_speed REAL,
    daily_snowfall REAL
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    trip_distance REAL,
    rate_code_id INTEGER,
    pickup_location_id INTEGER,
    dropoff_location_id INTEGER,
    base_passenger_fare REAL,
    rush_hour_surcharge REAL,
    mta_tax REAL,
    tip_amount REAL,
    tolls_amount REAL,
    improvement_surcharge REAL,
    total_amount REAL,
    congestion_surcharge REAL,
    airport_fee REAL
);
"""

# might not need request_datetime
UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips (
    request_datetime DATETIME,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    trip_distance REAL,
    pickup_location_id INTEGER,
    dropoff_location_id INTEGER,
    base_passenger_fare REAL,
    tip_amount REAL,
    tolls_amount REAL,
    black_car_fund_fee REAL,
    total_amount REAL,
    sales_tax, REAL
    congestion_surcharge REAL,
    airport_fee REAL
);
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    with open(DATABASE_SCHEMA_FILE, 'r') as schema_file:
        schema = schema_file.read()
        connection.execute(db.text(schema))

### Add Data to Database

In [ ]:
# final_taxi_data, uber_data, hourly_weather_data, daily_weather_data

# Load sampled trip data into each database
final_taxi_data.to_sql('taxi_trips', con=engine, if_exists='replace', index=False)
final_uber_data.to_sql('uber_trips', con=engine, if_exists='replace', index=False)
hourly_weather_data.to_sql('hourly_weather', con=engine, if_exists='replace', index=False)
daily_weather_data.to_sql('daily_weather', con=engine, if_exists='replace', index=False)


In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [ ]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [ ]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_1)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_1, con=engine)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)